# Mempersiapkan Dataset

In [ ]:
import os
import random

def generate_random_sample(base_folder):
    # Inisialisasi daftar untuk menyimpan path gambar yang terpilih
    gambar_terpilih = []

    # Membuat kamus untuk mengelompokkan nama file berdasarkan nama kelompok
    kelompok_dict = {}

    # Loop melalui semua file dalam direktori
    for root, _, files in os.walk(base_folder):
        for file_name in files:
            # Memisahkan nama file dari ekstensinya
            nama_file, ext = os.path.splitext(file_name)

            # Membagi nama file menjadi komponen berdasarkan _
            components = nama_file.split(' ')[1:-1]

            # Menggunakan komponen pertama sebagai nama kelompok
            kelompok = ' '.join(components)

            # Menambahkan path lengkap ke file ke dalam kamus kelompok
            if kelompok in kelompok_dict:
                kelompok_dict[kelompok].append(os.path.join(root, file_name))
            else:
                kelompok_dict[kelompok] = [os.path.join(root, file_name)]

    for kelompok in kelompok_dict:
        files_in_kelompok = kelompok_dict[kelompok]

        if len(files_in_kelompok) >= 10: #cek jumlah data setiap kelompok 
            two_random_files = random.sample(files_in_kelompok, 34) #34 berasal dari 30% dari semua data (dalam project ini ada 116 data)
            gambar_terpilih.extend(two_random_files)
            
    print(len(gambar_terpilih))

    return gambar_terpilih

In [ ]:
# Path ke direktori yang berisi gambar
base_folder = 'C:.../...' #sesuaikan dengan path folder dataset
gambar_terpilih = generate_random_sample(base_folder)

# Copy Hasil Images Terpilih ke Dalam Folder Testing


In [ ]:
import shutil

def copy_files(list_sumber_dir, tujuan_dir):
  '''
  Cek apakah direktori tujuan sudah ada,
  Jika sudah ada dan ada isinya, hapus direktori tersebut
  karena ingin membuat directori kosong
  jika belum, buat direktori tersebut
  '''
  if os.path.exists(tujuan_dir):
    shutil.rmtree(tujuan_dir)

  if not os.path.exists(tujuan_dir):
    os.makedirs(tujuan_dir)

  for i in range(0, len(list_sumber_dir)):
    shutil.copy(list_sumber_dir[i], tujuan_dir)

  print("Proses menyalin images selesai.")

In [ ]:
list_sumber_dir = gambar_terpilih
tujuan_dir = "C:.../..." #sesuaikan dengan path direktori anda

copy_files(list_sumber_dir, tujuan_dir)

# Buat Data Training

In [ ]:
testing = tujuan_dir

# Tentukan Data Training Selain Images yg Terpilih sebagai Data Testing
all_files = os.listdir(base_folder)
testing = os.listdir(testing)
print("Jumlah Data Testing: " + str(len(testing)))

training = [x for x in all_files if x not in testing]
print("Jumlah Data Training: " + str(len(training)))

# Buat List Path Images untuk Training
training_path = []
for root, _, files in os.walk(base_folder):
  for file_name in training:
    training_path.append(os.path.join(root, file_name))

training_path

In [ ]:
new_list_sumber_dir = training_path
new_tujuan_dir = "C:.../..." #sesuaikan dengan path direktori anda
# copy_files(new_list_sumber_dir, new_tujuan_dir)

# Pyfeats Textural Feature

In [ ]:
'''
REQUIREMENTS:
- Python version                     3.8.5
- opencv-python                      4.7.0.72
- matplotlib                         3.3.2
- pandas                             1.1.1
- pyfeats                            1.0.0
- os
- re
'''

# Import Images

In [ ]:
import cv2
import os
import re

def import_images(folder_path):
    # Buat list kosong buat penampung images dan label images
    images = []
    labels = []
    # inisialiasi bentuk regex
    pattern = r'[ _]+'
    # Memeriksa apakah folder tersebut ada
    if os.path.exists(folder_path):
        # Mendapatkan daftar file dalam folder
        files = os.listdir(folder_path)
        # Loop melalui setiap file dalam folder
        for file_name in files:
            # Memeriksa apakah file tersebut adalah file gambar (misalnya, dengan ekstensi .jpg atau .png)
            if file_name.endswith(('.jpg', '.png', '.jpeg')):
                # Menggabungkan path lengkap ke file gambar
                image_path = os.path.join(folder_path, file_name)
                file_name = os.path.basename(image_path)
                # Mencari semua kata dalam nama file
                words = re.findall(r'\w+', file_name)
                # Menggabungkan kata-kata tersebut menjadi satu string
                result = ' '.join(words[0:-2])
                # result = ' '.join(words[1:-2]) # Mengabaikan kata pertama (angka) dan dua kata terakhir (n_rotate_a dan ekstensi)
                labels.append(result)
                # Membaca gambar menggunakan OpenCV
                image = cv2.imread(image_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert ke grayscale
                images.append(gray)

            else:
                print(f"Ignoring non-image file: {file_name}")

    else:
        print(f"Folder not found: {folder_path}")

    print(len(images))
    print(len(labels))

    return images, labels

In [ ]:
# Data Training
folder_path_training = r"C:.../..." # Path ini diganti tergantung direktori tempat simpan filenya
images_training, labels_training = import_images(folder_path_training)

# Data Testing
folder_path_testing = r"C:.../..." # Path ini diganti tergantung direktori tempat simpan filenya
images_testing, labels_testing = import_images(folder_path_testing)

# Show Image

In [ ]:
import matplotlib.pyplot as plt

def show_image(images, labels):
  plt.imshow(images[0], cmap='gray') # Tampilkan citra pada list images index 0
  plt.title(labels[0])
  plt.show()

In [ ]:
# Data Training
show_image(images_training, labels_training)

In [ ]:
# Data Testing
show_image(images_testing, labels_testing)

# Pyfeat textural extraction

In [ ]:
!pip install pyfeats

In [ ]:
from pyfeats import *

class Features():
    def __init__(self):
        self.GLSZM = []
        self.Labels_GLSZM = []

def extract_texture(images, mask = None):
  GLSZM = []

  for f in images:
    features_GLSZM, labels_GLSZM = glszm_features(f, mask)
    GLSZM.append(features_GLSZM)

  # Panggil Class Features untuk membuat dan mengisi kamus features
  features = Features()
  features.GLSZM = GLSZM
  features.Labels_GLSZM = labels_GLSZM

  return features

In [ ]:
# Run function above
# Training
features_training = extract_texture(images_training)
# Testing
features_testing = extract_texture(images_testing)

# Transform ke Dataframe

In [ ]:
import pandas as pd

class Dataframes():
    #structure of the solution
    def __init__(self):
      self.GLSZM = None
      self.Class = None

def convert2dataframe(features, labels):
  df_GLSZM = pd.DataFrame(features.GLSZM, columns=features.Labels_GLSZM)
  df_class = pd.DataFrame(labels, columns=["Class"])

  dataframe = Dataframes()
  dataframe.GLSZM = df_GLSZM
  dataframe.Class = df_class

  return dataframe

In [ ]:
# Run function above
# Training
Dataframes_train = convert2dataframe(features_training, labels_training)
# Testing
Dataframes_test = convert2dataframe(features_testing, labels_testing)

# Dataframe GLSZM

In [ ]:
Dataframes_train.GLSZM

In [ ]:
Dataframes_test.GLSZM

# Dataframe Class




In [ ]:
Dataframes_train.Class

In [ ]:
Dataframes_test.Class

# Gabungkan Semua Dataframe

In [ ]:
df_result_training = pd.concat([Dataframes_train.GLSZM,
                                Dataframes_train.Class], axis=1)
# df_result_training

In [ ]:
df_result_testing = pd.concat([Dataframes_test.GLSZM,
                               Dataframes_test.Class], axis=1)
# df_result_testing

# Save Dataframe

In [ ]:
df_result_training.to_csv("C:.../.../Datasets Training GLSZM.csv", index=False) #sesuaikan dengan path direktori anda
df_result_testing.to_csv("C:.../.../Datasets Testing GLSZM.csv", index=False) #sesuaikan dengan path direktori anda